<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#LinkNet


In [ ]:
!pip install tensorflow
!pip install --quiet segmentation-models tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 593.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import keras
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    MaxPooling2D, Conv2DTranspose, concatenate
)
from tensorflow.keras.models import Model

import segmentation_models as sm

sm.set_framework('tf.keras')
sm.framework()

def build_linknet(backbone_name, input_shape, n_classes):
    """
    Builds a LinkNet model with the given backbone and freezes its encoder.
    """
    model = sm.Linknet(
        backbone_name,
        input_shape=input_shape,
        classes=n_classes,
        activation='softmax',
        encoder_weights='imagenet'
    )
    # Freeze encoder layers
    for layer in model.layers:
        if 'encoder' in layer.name:
            layer.trainable = False
    return model

# Example instantiation with EfficientNetB7
linknet = build_linknet(
    backbone_name='efficientnetb7',
    input_shape=(512, 512, 3),
    n_classes=5
)

linknet.summary()

# 2) List every layer’s name and its output shape—even InputLayers
for i, layer in enumerate(linknet.layers):
    # InputLayer doesn’t have .output_shape, but it does have .output
    shape = getattr(layer, 'output_shape', None) or tuple(layer.output.shape)
    print(f"{i:03d} | {layer.name:30s} | {shape}")




Segmentation Models: using `tf.keras` framework.
258434480/258434480 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 256, 256,  │      1,728 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 256, 256,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 256, 256,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 256, 256,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 256, 256,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 256, 256,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 256, 256,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 256, 256,  │      2,048 │ block1a_se_excit… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 256, 256,  │        128 │ block1a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_dwconv      │ (None, 256, 256,  │        288 │ block1a_project_… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_bn          │ (None, 256, 256,  │        128 │ block1b_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_activation  │ (None, 256, 256,  │          0 │ block1b_bn[0][0]

 Total params: 72,264,997 (275.67 MB)

 Trainable params: 71,944,773 (274.45 MB)

 Non-trainable params: 320,224 (1.22 MB)

000 | input_layer                    | (None, 512, 512, 3)
001 | stem_conv                      | (None, 256, 256, 64)
002 | stem_bn                        | (None, 256, 256, 64)
003 | stem_activation                | (None, 256, 256, 64)
004 | block1a_dwconv                 | (None, 256, 256, 64)
005 | block1a_bn                     | (None, 256, 256, 64)
006 | block1a_activation             | (None, 256, 256, 64)
007 | block1a_se_squeeze             | (None, 64)
008 | block1a_se_reshape             | (None, 1, 1, 64)
009 | block1a_se_reduce              | (None, 1, 1, 16)
010 | block1a_se_expand              | (None, 1, 1, 64)
011 | block1a_se_excite              | (None, 256, 256, 64)
012 | block1a_project_conv           | (None, 256, 256, 32)
013 | block1a_project_bn             | (None, 256, 256, 32)
014 | block1b_dwconv                 | (None, 256, 256, 32)
015 | block1b_bn                     | (None, 256, 256, 32)
016 | block1b_activation             | (None, 256, 256, 32)
017

In [ ]:
'''from tensorflow.keras.models import Model

# 1) Build the full LinkNet as usual
linknet = build_linknet(
    backbone_name='efficientnetb7',
    input_shape=(512,512,3),
    n_classes=5
)

# 2) Grab the bottleneck output
bottleneck = linknet.get_layer("top_activation").output

# 3) Wrap it in its own Model
encoder = Model(inputs=linknet.input, outputs=bottleneck, name="LinkNet_Encoder")

# 4) Sanity check
encoder.summary()
# You should see: Output shape (None, 7, 7, 2560)
'''

'from tensorflow.keras.models import Model\n\n# 1) Build the full LinkNet as usual\nlinknet = build_linknet(\n    backbone_name=\'efficientnetb7\',\n    input_shape=(512,512,3),\n    n_classes=5\n)\n\n# 2) Grab the bottleneck output\nbottleneck = linknet.get_layer("top_activation").output\n\n# 3) Wrap it in its own Model\nencoder = Model(inputs=linknet.input, outputs=bottleneck, name="LinkNet_Encoder")\n\n# 4) Sanity check\nencoder.summary()\n# You should see: Output shape (None, 7, 7, 2560)\n'

# Metrics

In [ ]:
import tensorflow as tf

smooth = 1e-15
num_classes = 5  # <-- make this match your final Conv2D(filters=5, ...)

def dice_coeff(y_true, y_pred):
    # y_true: (B, H, W, 1) integers 0..4
    # y_pred: (B, H, W, 5) softmax probs

    # 1) Squeeze off the last dim → (B, H, W)
    y_true = tf.squeeze(y_true, axis=-1)

    # 2) One‑hot encode → (B, H, W, 5)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

    # 3) Flatten both to (B*H*W, 5)
    y_true_f = tf.reshape(y_true_o, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred,   [-1, num_classes])

    # 4) Compute intersection & union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union        = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)

    # 5) Dice per class, then average
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

#Train


In [ ]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam

""" Global parameters """
Height = 512
Width  = 512

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask

def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 28.2 MB/s eta 0:00:00
Train: 1216, Val: 348, Test: 174


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)

# Hyper‑params & paths
NUM_CLASSES = 5
INPUT_SHAPE = (512, 512, 3)
EPOCHS      = 160
path_model  = "/content/drive/MyDrive/Project_41725/files/model.h5"
path_csv    = "/content/drive/MyDrive/Project_41725/files/log.csv"

# 0) kill any existing graph/state
tf.keras.backend.clear_session()

# 1) rebuild
model = build_linknet(
    backbone_name='efficientnetb7',
    input_shape=(512, 512, 3),
    n_classes=5
)

# 2) override its internal name (strip out the '+')
model._name = "linknet"

# 3) Define losses & metrics
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)


#Test


In [ ]:
pip install tensorflow opencv-python-headless matplotlib

In [ ]:
# Install Gradio
!pip install -q gradio

# Imports
import gradio as gr
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# ─── 1) Load Model ────────────────────────────────────────────────────────
MODEL_PATH = "/content/drive/MyDrive/Project_41725/files/LinkNet/model.h5"
model = load_model(MODEL_PATH, compile=False)

# Automatically get model input dimensions
H_IN, W_IN = model.input_shape[1:3]

# ─── 2) Define Class Names & Colors ────────────────────────────────────────
CLASS_NAMES = [
    "Background",
    "g",
    "m",
    "p",
    "D",
]
CLASS_COLORS = [
    (  0,   0,   0),  # black for Background
    (255,   0,   0),  # red   for Tumor Core
    (  0, 255,   0),  # green for Enhancing Tumor
    (  0,   0, 255),  # blue  for Edema
    (255, 255,   0),  # yellow for Necrosis
]

# Build an HTML legend
legend_html = "<div style='display:flex; gap:1rem; flex-wrap:wrap; margin-bottom:1em;'>"
for name, (r, g, b) in zip(CLASS_NAMES, CLASS_COLORS):
    legend_html += f"""
      <div style='display:flex; align-items:center; gap:0.3rem;'>
        <div style='width:20px; height:20px; background: rgb({r},{g},{b}); border:1px solid #333;'></div>
        <span style='font-size:14px;'>{name}</span>
      </div>
    """
legend_html += "</div>"

# ─── 3) Define Prediction and Visualization Function ────────────────────────
def predict_mask_colored(img: np.ndarray):
    # img: H×W×3 RGB input
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    orig_h, orig_w = gray.shape

    # Resize to model input dimensions
    small = cv2.resize(gray, (W_IN, H_IN), interpolation=cv2.INTER_AREA)
    x = np.stack([small] * 3, axis=-1).astype(np.float32) / 255.0

    # Predict segmentation mask
    pred = model.predict(x[None], verbose=0)[0]   # (H_IN, W_IN, C)
    seg = pred.argmax(-1).astype(np.uint8)        # Predicted class IDs

    # Upsample back to original image size
    seg_up = cv2.resize(seg, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

    # Create colored mask
    color_mask = np.zeros((orig_h, orig_w, 3), dtype=np.uint8)
    for cls_id, color in enumerate(CLASS_COLORS):
        color_mask[seg_up == cls_id] = color

    # Create overlay
    overlay = cv2.addWeighted(img, 0.6, color_mask, 0.4, 0)

    return color_mask, overlay

# ─── 4) Build Gradio App ────────────────────────────────────────────────────
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🖥️ Brain MRI Segmentation Demo")
    gr.HTML(legend_html)

    with gr.Row():
        inp = gr.Image(type="numpy", label="Upload Brain MRI Image")
        with gr.Column():
            out1 = gr.Image(type="numpy", label="Predicted Color Mask")
            out2 = gr.Image(type="numpy", label="Overlay on Original Image")

    btn = gr.Button("Segment", variant="primary")
    btn.click(predict_mask_colored, inputs=inp, outputs=[out1, out2])

    gr.Markdown(
        "<p style='font-size:12px;color:#666;'>"
        "Each abnormality class is shown with a different color for easy interpretation."
        "</p>"
    )

# ─── 5) Launch App ─────────────────────────────────────────────────────────
demo.launch(share=True, debug=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 132.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://72a737db6ff46b0f3a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging F